In [1]:
import pandas as pd

In [2]:
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

In [3]:
merged_df = pd.merge(transactions_df, customers_df, on='CustomerID', how='inner')
merged_df = pd.merge(merged_df, products_df, on='ProductID', how='inner')

In [4]:
merged_df.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [5]:
#Extracting customer-level features

customer_features = merged_df.groupby('CustomerID').agg(
    total_spent=('TotalValue', 'sum'),
    avg_spent_per_transaction=('TotalValue', 'mean'),
    transaction_count=('TransactionID', 'count'),
    unique_products_bought=('ProductID', 'nunique')
).reset_index()

In [6]:
#Adding demographic information

customer_features = pd.merge(customer_features, customers_df[['CustomerID', 'Region']], on='CustomerID', how='left')

In [7]:
merged_df['SignupDate'] = pd.to_datetime(merged_df['SignupDate'])
merged_df['current_date'] = pd.to_datetime('today')
merged_df['tenure'] = (merged_df['current_date'] - merged_df['SignupDate']).dt.days / 365
customer_features['tenure'] = merged_df.groupby('CustomerID')['tenure'].max().reset_index()['tenure']

In [8]:
customer_features.head()

,CustomerID,total_spent,avg_spent_per_transaction,transaction_count,unique_products_bought,Region,tenure
0,C0001,3354.52,670.904,5,5,South America,2.556164
1,C0002,1862.74,465.685,4,4,Asia,2.958904
2,C0003,2725.38,681.345,4,4,South America,0.895890
3,C0004,5354.88,669.360,8,8,South America,2.306849
4,C0005,2034.24,678.080,3,3,Asia,2.457534


In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
scaler = StandardScaler()
features = customer_features[['total_spent', 'avg_spent_per_transaction', 'transaction_count', 'unique_products_bought', 'tenure']]
scaled_features = scaler.fit_transform(features)

In [11]:
similarity_matrix = cosine_similarity(scaled_features)

In [12]:
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])
similarity_df.head()

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.784652,-0.676519,0.358147,0.572379,-0.363411,0.588739,-0.117600,0.025922,0.445791,...,-0.826646,0.654412,0.549956,-0.481546,-0.858331,0.344696,0.246860,0.545044,0.522145,0.009493
C0002,0.784652,1.000000,-0.162996,-0.135513,0.814256,-0.584228,0.591336,-0.303836,0.639205,0.901907,...,-0.370886,0.980302,0.616083,-0.522402,-0.844350,-0.051178,0.677613,0.902149,0.938016,-0.524414
C0003,-0.676519,-0.162996,1.000000,-0.917566,0.206465,0.302142,0.060235,-0.500301,0.569615,0.192906,...,0.702662,0.034016,0.140207,-0.158469,0.241945,-0.224591,0.543300,0.226413,0.162864,-0.302416
C0004,0.358147,-0.135513,-0.917566,1.000000,-0.560548,-0.322322,-0.434957,0.778743,-0.653639,-0.383297,...,-0.393371,-0.314621,-0.508499,0.525199,0.154194,-0.027255,-0.807916,-0.537823,-0.397674,0.251041
C0005,0.572379,0.814256,0.206465,-0.560548,1.000000,-0.037584,0.905031,-0.787318,0.595827,0.731197,...,-0.387283,0.856132,0.935159,-0.887802,-0.896163,0.329867,0.928479,0.954870,0.811693,-0.211064


In [13]:
lookalike_dict = {}

In [14]:
for customer_id in customer_features['CustomerID'][:20]:
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:4]
    lookalike_dict[customer_id] = [(cust, similar_customers[cust]) for cust in similar_customers.index]

In [15]:
lookalike_dict

{'C0001': [('C0152', 0.999289866799282),
  ('C0160', 0.964655879087813),
  ('C0134', 0.9311918572294455)],
 'C0002': [('C0029', 0.9957010276321365),
  ('C0192', 0.9803019004008591),
  ('C0025', 0.9677000847652859)],
 'C0003': [('C0036', 0.9822843560263196),
  ('C0177', 0.9759120148186363),
  ('C0144', 0.9736705930538494)],
 'C0004': [('C0175', 0.9980347719453085),
  ('C0173', 0.9915760332813376),
  ('C0108', 0.985499229820579)],
 'C0005': [('C0073', 0.9997632359260057),
  ('C0159', 0.9994013848987509),
  ('C0112', 0.9973831126489384)],
 'C0006': [('C0066', 0.9631264545710321),
  ('C0185', 0.9342364869237268),
  ('C0044', 0.9113169980270031)],
 'C0007': [('C0193', 0.9946703663714069),
  ('C0125', 0.9898186028592005),
  ('C0176', 0.976151210699048)],
 'C0008': [('C0090', 0.9907154142541296),
  ('C0017', 0.9713096204441193),
  ('C0024', 0.9583394457397199)],
 'C0009': [('C0077', 0.9969180110848981),
  ('C0097', 0.9786014665414042),
  ('C0094', 0.9673125156146763)],
 'C0010': [('C0083', 0.

In [16]:
lookalike_list = []
for cust_id, lookalikes in lookalike_dict.items():
    for similar_cust, score in lookalikes:
        lookalike_list.append([cust_id, similar_cust, score])

lookalike_df = pd.DataFrame(lookalike_list, columns=['CustomerID', 'LookalikeCustomerID', 'SimilarityScore'])

In [17]:
lookalike_df.to_csv('Lookalike.csv', index=False)

lookalike_df.head()

,CustomerID,LookalikeCustomerID,SimilarityScore
0,C0001,C0152,0.999290
1,C0001,C0160,0.964656
2,C0001,C0134,0.931192
3,C0002,C0029,0.995701
4,C0002,C0192,0.980302
